# 1. Data Preprocessing


In [1]:
import pandas as pd
from sklearn.utils import shuffle

!pip install text_hammer 
import text_hammer as th
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
def text_preprocessing(df,col_name):
    column = col_name
    df[column] = df[column].progress_apply(lambda x:str(x).lower())
#     df[column] = df[column].progress_apply(lambda x: th.cont_exp(x))
    #you're -> you are; i'm -> i am
    df[column] = df[column].progress_apply(lambda x: th.remove_emails(x))
    df[column] = df[column].progress_apply(lambda x: th.remove_html_tags(x))
    df[column] = df[column].progress_apply(lambda x: th.remove_special_chars(x))
    df[column] = df[column].progress_apply(lambda x: th.remove_accented_chars(x))
    df[column] = df[column].progress_apply(lambda x: th.make_base(x)) #ran -> run,
    return(df)

df = pd.read_csv('/kaggle/input/twitter-and-reddit-sentimental-analysis-dataset/Reddit_Data.csv')
df.dropna(inplace = True)
df.rename(columns = {'clean_comment':'text', 'category':'sentiment'}, inplace = True)
df = shuffle(df)
df = text_preprocessing(df, 'text')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.8 MB/s eta 0:00:0000:01
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.11.1
    Uninstalling beautifulsoup4-4.11.1:
      Successfully uninstalled beautifulsoup4-4.11.1


  0%|          | 0/37149 [00:00<?, ?it/s]

  0%|          | 0/37149 [00:00<?, ?it/s]

  0%|          | 0/37149 [00:00<?, ?it/s]

  0%|          | 0/37149 [00:00<?, ?it/s]

  0%|          | 0/37149 [00:00<?, ?it/s]

  0%|          | 0/37149 [00:00<?, ?it/s]

In [2]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer=Tokenizer(num_words= 10000,lower=True)
num_words = 10000
X = df['text']
Y = df['sentiment']
 # this means 10000 unique words can be taken 
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X) # this converts texts into some numeric sequences 
X = pad_sequences(X,maxlen=100,padding='post') # this makes the length of all numeric sequences equal 


#Splitting the data into training and testing
from sklearn.model_selection import train_test_split
y=pd.get_dummies(df['sentiment'])
X_train, X_valid_test, y_train, y_valid_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
X_valid, X_test, y_valid, y_test = train_test_split(X_valid_test, y_valid_test, test_size = 0.66, random_state = 42)

# 2. Model Training And Testing

In [3]:
import tensorflow
import numpy as np
from sklearn import metrics
from tensorflow.keras.models import Sequential 
from keras.layers import Flatten
from tensorflow.keras.layers import Dense, LSTM, Embedding,Bidirectional
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.layers import Dropout

def traing(model):
    es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 3)
    mc = ModelCheckpoint('./'+str(model)+'-model.h5', monitor = 'val_accuracy', mode = 'max', verbose = 1, save_best_only = True)
    history_embedding = model.fit(X_train, y_train, 
                                    epochs = 5, batch_size = 64, 
                                    validation_data=(X_valid, y_valid),
                                    verbose = 1, callbacks= [es, mc]  )
    return model

def testing(model):
    y_pred =  np.argmax(model.predict(X_test), axis  =  1)
    y_true = np.argmax(y_test.values, axis = 1)
    print(metrics.classification_report(y_pred, y_true, target_names=['negtive','neutral','positive']))

# (1) Bi-LSTM

In [4]:
def get_BiLSTM():
    EMBEDDING_DIM = 100
    model = Sequential()
    model.add(Embedding(input_dim = num_words,
     output_dim = EMBEDDING_DIM,
     input_length= X.shape[1]))
    model.add(Bidirectional(CuDNNLSTM(100,return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(CuDNNLSTM(200,return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(100,activation = 'relu'))
    model.add(Dense(3, activation = 'softmax'))
    model.compile(loss = 'mean_squared_error', optimizer = 'sgd',metrics = 'accuracy')
    return model

In [5]:
lstm_model = traing(get_BiLSTM())

Epoch 1/5
407/407 [==============================] - ETA: 0s - loss: 0.2129 - accuracy: 0.4668
Epoch 1: val_accuracy improved from -inf to 0.58168, saving model to ./<keras.engine.sequential.Sequential object at 0x7a38ce52a950>-model.h5
407/407 [==============================] - 53s 107ms/step - loss: 0.2129 - accuracy: 0.4668 - val_loss: 0.2052 - val_accuracy: 0.5817
Epoch 2/5
407/407 [==============================] - ETA: 0s - loss: 0.1984 - accuracy: 0.5640
Epoch 2: val_accuracy did not improve from 0.58168
407/407 [==============================] - 27s 66ms/step - loss: 0.1984 - accuracy: 0.5640 - val_loss: 0.1898 - val_accuracy: 0.5809
Epoch 3/5
407/407 [==============================] - ETA: 0s - loss: 0.1880 - accuracy: 0.5650
Epoch 3: val_accuracy did not improve from 0.58168
407/407 [==============================] - 22s 55ms/step - loss: 0.1880 - accuracy: 0.5650 - val_loss: 0.1833 - val_accuracy: 0.5804
Epoch 4/5
407/407 [==============================] - ETA: 0s - loss: 0.

In [6]:
testing(lstm_model)

230/230 [==============================] - 4s 13ms/step
              precision    recall  f1-score   support

     negtive       0.00      0.00      0.00         0
     neutral       0.72      0.61      0.66      3000
    positive       0.75      0.55      0.63      4356

    accuracy                           0.57      7356
   macro avg       0.49      0.39      0.43      7356
weighted avg       0.74      0.57      0.64      7356



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# (2) CNN

In [7]:
from tensorflow.keras.layers import Input,Conv1D,MaxPooling1D,Dense,GlobalMaxPooling1D,Embedding
from tensorflow.keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten
from keras.preprocessing import sequence

In [8]:
def get_CNN():
    EMBEDDING_DIM = 100
    model = Sequential()
    model.add(Embedding(input_dim = num_words,
     output_dim = EMBEDDING_DIM,
     input_length= X.shape[1]))
    model.add(Conv1D(128, 3, activation = 'relu'))
    model.add(MaxPooling1D(3))
    model.add(Conv1D(64,3,activation = 'relu'))
    model.add(Flatten())
    model.add(Dense(250,activation = 'relu'))
    model.add(Dense(3, activation = 'softmax'))
    model.compile(loss = 'mean_squared_error', optimizer = 'sgd',metrics = 'accuracy')
    return model

In [9]:
cnn_model = traing(get_CNN())

Epoch 1/5
407/407 [==============================] - ETA: 0s - loss: 0.2175 - accuracy: 0.4130
Epoch 1: val_accuracy improved from -inf to 0.42307, saving model to ./<keras.engine.sequential.Sequential object at 0x7a37104efe50>-model.h5
407/407 [==============================] - 16s 31ms/step - loss: 0.2175 - accuracy: 0.4130 - val_loss: 0.2131 - val_accuracy: 0.4231
Epoch 2/5
407/407 [==============================] - ETA: 0s - loss: 0.2106 - accuracy: 0.4686
Epoch 2: val_accuracy improved from 0.42307 to 0.57984, saving model to ./<keras.engine.sequential.Sequential object at 0x7a37104efe50>-model.h5
407/407 [==============================] - 6s 15ms/step - loss: 0.2106 - accuracy: 0.4686 - val_loss: 0.2064 - val_accuracy: 0.5798
Epoch 3/5
407/407 [==============================] - ETA: 0s - loss: 0.2033 - accuracy: 0.5710
Epoch 3: val_accuracy did not improve from 0.57984
407/407 [==============================] - 5s 12ms/step - loss: 0.2033 - accuracy: 0.5710 - val_loss: 0.1976 - v

In [10]:
testing(cnn_model)

230/230 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

     negtive       0.00      0.00      0.00         0
     neutral       0.83      0.56      0.67      3793
    positive       0.65      0.58      0.62      3563

    accuracy                           0.57      7356
   macro avg       0.50      0.38      0.43      7356
weighted avg       0.75      0.57      0.64      7356



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# (3) LSTM-CNN

In [11]:
def get_LSTM_CNN():
    EMBEDDING_DIM = 100
    model = Sequential()
    model.add(Embedding(input_dim = num_words,
     output_dim = EMBEDDING_DIM,
     input_length= X.shape[1]))
    model.add(Dropout(0.2))
    model.add(Bidirectional(CuDNNLSTM(100,return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(CuDNNLSTM(200,return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Conv1D(128, 3, activation = 'relu'))
    model.add(MaxPooling1D(3))
    model.add(Conv1D(64,3,activation = 'relu'))
    model.add(Flatten())
    model.add(Dense(250,activation = 'relu'))
    model.add(Dense(3, activation = 'softmax'))
    model.compile(loss = 'mean_squared_error', optimizer = 'sgd',metrics = 'accuracy')
    return model

In [12]:
lstm_cnn_model = traing(get_LSTM_CNN())

Epoch 1/5
407/407 [==============================] - ETA: 0s - loss: 0.2164 - accuracy: 0.4351
Epoch 1: val_accuracy improved from -inf to 0.42307, saving model to ./<keras.engine.sequential.Sequential object at 0x7a3710260a50>-model.h5
407/407 [==============================] - 31s 66ms/step - loss: 0.2164 - accuracy: 0.4351 - val_loss: 0.2127 - val_accuracy: 0.4231
Epoch 2/5
407/407 [==============================] - ETA: 0s - loss: 0.2112 - accuracy: 0.4492
Epoch 2: val_accuracy improved from 0.42307 to 0.49010, saving model to ./<keras.engine.sequential.Sequential object at 0x7a3710260a50>-model.h5
407/407 [==============================] - 23s 56ms/step - loss: 0.2112 - accuracy: 0.4492 - val_loss: 0.2079 - val_accuracy: 0.4901
Epoch 3/5
407/407 [==============================] - ETA: 0s - loss: 0.2044 - accuracy: 0.5500
Epoch 3: val_accuracy improved from 0.49010 to 0.56532, saving model to ./<keras.engine.sequential.Sequential object at 0x7a3710260a50>-model.h5
407/407 [========

In [13]:
testing(lstm_cnn_model)

230/230 [==============================] - 4s 13ms/step
              precision    recall  f1-score   support

     negtive       0.00      0.00      0.00         0
     neutral       0.86      0.54      0.67      4035
    positive       0.62      0.59      0.60      3321

    accuracy                           0.56      7356
   macro avg       0.49      0.38      0.42      7356
weighted avg       0.75      0.56      0.64      7356



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# (4) CNN-LSTM

In [14]:
def get_CNN_LSTM():
    EMBEDDING_DIM = 100
    model = Sequential()
    model.add(Embedding(input_dim = num_words,
     output_dim = EMBEDDING_DIM,
     input_length= X.shape[1]))
    model.add(Conv1D(256, 3, activation = 'relu'))
    model.add(MaxPooling1D(3))
    model.add(Conv1D(128,3,activation = 'relu'))
    model.add(Dropout(0.2))
    model.add(CuDNNLSTM(64,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(CuDNNLSTM(128,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(200,activation = 'relu'))
    model.add(Dense(3, activation = 'softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',metrics = 'accuracy')
    return model

In [15]:
cnn_lstm_model = traing(get_CNN_LSTM())

Epoch 1/5
407/407 [==============================] - ETA: 0s - loss: 0.6514 - accuracy: 0.7238
Epoch 1: val_accuracy improved from -inf to 0.80892, saving model to ./<keras.engine.sequential.Sequential object at 0x7a36ecedfed0>-model.h5
407/407 [==============================] - 61s 138ms/step - loss: 0.6514 - accuracy: 0.7238 - val_loss: 0.4865 - val_accuracy: 0.8089
Epoch 2/5
407/407 [==============================] - ETA: 0s - loss: 0.3248 - accuracy: 0.8860
Epoch 2: val_accuracy improved from 0.80892 to 0.88044, saving model to ./<keras.engine.sequential.Sequential object at 0x7a36ecedfed0>-model.h5
407/407 [==============================] - 18s 44ms/step - loss: 0.3248 - accuracy: 0.8860 - val_loss: 0.3358 - val_accuracy: 0.8804
Epoch 3/5
407/407 [==============================] - ETA: 0s - loss: 0.2066 - accuracy: 0.9317
Epoch 3: val_accuracy did not improve from 0.88044
407/407 [==============================] - 13s 33ms/step - loss: 0.2066 - accuracy: 0.9317 - val_loss: 0.3605 

In [16]:
testing(cnn_lstm_model)

230/230 [==============================] - 1s 4ms/step
              precision    recall  f1-score   support

     negtive       0.77      0.74      0.75      1698
     neutral       0.88      0.91      0.89      2487
    positive       0.86      0.86      0.86      3171

    accuracy                           0.85      7356
   macro avg       0.84      0.84      0.84      7356
weighted avg       0.85      0.85      0.85      7356

